TRABAJAMOS SOBRE EL MOVIES_DATASET.CSV

In [2]:
# Importamos las librerías necesarias para poder trabajar con los datos
import pandas as pd
import numpy as np
import ast
from datetime import datetime
from sklearn.metrics.pairwise import sigmoid_kernel
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
# Importamos el dataset credits y los convertimos a dataframe para poder trabajar con él
df_movies = pd.read_csv("../datasets/raw/movies_dataset.csv", sep=",", low_memory=False)

In [4]:
# Observando los datos anteriores decido trabajar sobre estas columnas
columns_to_transform = [
    "belongs_to_collection",
    "genres",
    "production_companies",
    "production_countries",
    "spoken_languages",
    "id"
]

In [5]:
# Trabajamos sobre la columna belongs_to_collection
df = df_movies  # Crea un dataframe para trabajar sobre este y no sobre el original

In [6]:
# Borramos las filas duplicadas en base al id
df.drop_duplicates(subset='id', keep='first', inplace=True)

In [7]:


# Creamos una funcion para hacer una evaluacion literal de las filas que se le asigne
def transform_row(row):
    """
    Esta funcion toma una fila, si es de tipo string que contiene una representación literal de un objeto de datos de Python,
     como un diccionario, una lista, una tupla, un conjunto, un número o una cadena, y la interpreta como el objeto real en memoria.
    """
    if isinstance(row, str):  # Checkeamos si la fila es de formato string
        try:
            evaluation = ast.literal_eval(row)  # Convertimos el string a diccionario
            return evaluation
        except (SyntaxError, ValueError):
            return None  # Devuelve None si el string no puede ser convertido en diccionario
    return np.nan  # Retorna NaN si es el valor es NaN

In [8]:
# Recorremos las columnas que decidimos transformar y aplica la funcion transform_now
for column in columns_to_transform:
    transformed_column = [
        transform_row(row) for row in df[column]
    ]  # Aplicamos la funcion en cada fila de la columna y la guarda en una variable en formato lista
    df[
        column
    ] = transformed_column  # Guardamos la lista creada anteriormente a la columna belongs_to_collection

In [9]:
# Rellenamos los valores NaN por el valor 0 de las columnas revenue y budget
df["revenue"] = df["revenue"].fillna(0)
df["budget"] = df["budget"].fillna(0)
# Rellenamos los valores NaN por el string 0 de la columna overview porque después vamos a usar un modelo que no acepta enteros
df["overview"] = df["overview"].fillna('0')

In [10]:
# Removemos las filas con valores nulos en la columna release_date
# Iteramos sobre los valores de la columna release_date
for index, value in df["release_date"].items():
    if pd.isna(value):
        df.drop(index, inplace=True)

# Reseteamos los indices del dataframe
df.reset_index(drop=True, inplace=True)

In [11]:
# Reemplazamos no-infinitos valores por 0
df["release_date"].replace([np.nan, np.inf, -np.inf], 0, inplace=True)

# Encontramos los índices donde el valor es cero
indices_cero = df.loc[df["release_date"] == 0].index
print(indices_cero)
# Borramos las filas con los índices encontrados
df.drop(indices_cero, inplace=True)

# Reseteamos los indices del dataframe
df.reset_index(drop=True, inplace=True)

Int64Index([], dtype='int64')


In [12]:
# Transformamos la columna de fechas al formato AAAA-mm-dd
df["release_date"] = pd.to_datetime(
    df["release_date"], errors="coerce", format="%Y-%m-%d"
)
df["release_date"] = df["release_date"].dt.date

# Extraemos el año de la columna release_date y lo guarda en una nueva columna llamada release_year
df["release_year"] = pd.to_datetime(df["release_date"]).dt.year

# Reemplazamos no-infinitos valores por 0
df["release_year"].replace([np.nan, np.inf, -np.inf], 0, inplace=True)

# Encontramos los índices donde el valor es cero
indices_cero = df.loc[df["release_year"] == 0].index

# Borramos las filas con los índices encontrados
df.drop(indices_cero, inplace=True)

# Reseteamos los indices del dataframe
df.reset_index(drop=True, inplace=True)

# Convertimos la columna a números enteros
df["release_year"] = df["release_year"].astype(int)

In [13]:
# Creamos la columna return que se calcula como el revenue sobre budget

# Transformamos los datos a tipo float para poder usarlos en operaciones matemáticas
df["budget"] = df["budget"].astype(float)
df["revenue"] = df["revenue"].astype(float)

df["return"] = df["revenue"] / df["budget"]  # Calculamos el return

# Reemplazamos los NaN (ocurre cuando el valor de revenue es 0) y los infinitos (ocurre cuando el valor de buget es 0) por el valor 0.
df["return"].replace([np.nan, np.inf, -np.inf], 0, inplace=True)

In [14]:
# Eliminamos las columnas que no son reelevantes para este proyecto
columns_to_delete = [
    "video",
    "imdb_id",
    "adult",
    "original_title",
    "poster_path",
    "homepage",
    "belongs_to_collection",
    "genres",
    "original_language",
    "production_companies",
    "production_countries",
    "spoken_languages",
    "status",
    "tagline",
    "runtime"

]
df = df.drop(columns=columns_to_delete)

TRABAJAMOS SOBRE CREDITS.CSV

In [15]:
#Importamos el dataset credits y los convertimos a dataframe para poder trabajar con él
df_credits = pd.read_csv("../datasets/raw/credits.csv", sep=",")

In [16]:
def row_to_dict(df_name):
    """
        Esta funcion toma una fila, si es de tipo string que contiene una representación literal de un objeto de datos de Python,
     como un diccionario, una lista, una tupla, un conjunto, un número o una cadena, y la interpreta como el objeto real en memoria. Exceptúa la columna 'id'.

    """
    for column in df_name:
        if column == 'id':
            None
        else:
            # Evaluamos las filas del dataframe y las transformamos
            df = df_name
            df[column] = df_credits[column].apply(lambda x: eval(x))
            
    return df




In [17]:
# Evaluamos las filas del dataframe df_credits
df_credits_evaluated = row_to_dict(df_credits)

In [18]:
def unnest_columns(df,column,fixed_column,new_fixed_column_name):
    '''Esta función toma una columna (column) que contenga una lista de disccionarios, de un dataframe (df) y las desanida. 
    Crea un nuevo dataframe (new_df) tomando cada key del diccionario como nombre de columna en el nuevo dataframe y 
    dejando fijo el valor de una columna (fixed_column), nombrandola segun el input new_fixed_column_name.
    column, fixed_column y new_fixed_column_name deben ingresarse en formato string.
     '''
     # Creamos una nueva lista para almacenar los diccionarios desanidados
    diccionarios_desanidados = []

    # Iteramos sobre cada fila de la columna 'columna_lista'
    for lista_diccionarios,movieid in zip(df[column],df[fixed_column]):
        # Iteramos sobre cada diccionario en la lista
        for diccionario in lista_diccionarios:
            # Agregamos cada diccionario a la lista de diccionarios desanidados
            diccionario[new_fixed_column_name] = movieid
            diccionarios_desanidados.append(diccionario)
                    

    # Creamos un nuevo DataFrame con los diccionarios desanidados
    new_df = pd.DataFrame(diccionarios_desanidados)
    return new_df

In [19]:
# Desanidamos la columna cast del df_credits_evaluated y lo guardamos en el dataframe df_unnested_cast
df_unnested_cast = unnest_columns(df_credits_evaluated, 'cast', 'id', 'movie_id')

# Desanidamos la columna cast del df_credits_evaluated y lo guardamos en el dataframe df_unnested_crew
df_unnested_crew = unnest_columns(df_credits_evaluated, 'crew', 'id', 'movie_id')

In [20]:
# Eliminamos las columnas que no son reelevantes para este proyecto del dataframe cast
columns_to_delete_cast = [
    "cast_id",
    "character",
    "credit_id",
    "gender",
    "order",
    "profile_path",

]
df_cast = df_unnested_cast.drop(columns=columns_to_delete_cast)

In [21]:
#Filtramos las filas del df_unnested_crew donde el trabajo es director y lo guardamos en un dataframe llamado df_director
df_director = df_unnested_crew[df_unnested_crew['job'] == 'Director']  

#Eliminamos las columnas que no son reelevantes para este proyecto del dataframe df_director
columns_to_delete_director = [
    "credit_id",
    "department",
    "gender",
    "job",
    "profile_path",

]
df_director = df_director.drop(columns=columns_to_delete_director)

In [22]:
# Creamos un dataframe de muestra para reducir el tiempo de procesamiento ya que la capacidad de render es limitada
sample = df.sample(n=1000, random_state=42).reset_index(drop=True)  # n es el número de filas de la muestra

In [23]:
# Creamos los csv en base a los dataframes creado
df_cast.to_csv('../datasets/model/cast.csv', index=False)
df_director.to_csv('../datasets/model/director.csv', index=False)
df.to_csv('../datasets/model/movies.csv', index=False)
sample.to_csv('../datasets/model/sample.csv',index=False)